In [156]:
import pandas
import requests
import datetime
import time
import json
from dateutil import parser

In [157]:
#Different type of Ripple Transactions...
#https://ripple.com/build/transactions/
#denomination of XRP is 1,000,000 to 1

In [158]:
#https://ripple.com/build/data-api-v2/#get-transactions
url_endpoint = "https://data.ripple.com/v2/transactions/"
starting_datetime = datetime.datetime(2017,12,31,0,0)
iso_starting_datetime = starting_datetime.isoformat()
ending_timestamp = datetime.datetime(2017,12,31,1,0)
iso_ending_timestamp = ending_timestamp.isoformat()

In [159]:
#Datapoints per transaction needed
# Timestamp
# Unique Hash
# Account Sending
# Account Receiving (May be smart contract)
# Tx Fee
# Size of Data (I don't see this here... I'll skip it for now)
# Network Name (duh)
# Transaction Status
# Value of Tx

In [160]:
#Getting a test transaction
params = {"start": iso_starting_datetime , "end" : iso_ending_timestamp, "limit" : 100}
resp = requests.get(url_endpoint, params=params)

In [161]:
resp_dict = json.loads(resp.content)

In [162]:
marker = resp_dict['marker']
transactions = resp_dict['transactions']

In [165]:
# Timestamp
# Unique Hash
# Account Sending
# Account Receiving (May be smart contract)
# Tx Fee
# Size of Data (I don't see this here... I'll skip it for now)
# Network Name (duh)
# Transaction Status
# Value of Tx

processed_txs = []

for n, i in enumerate(transactions):
    tx_datetime = parser.parse(i['date'])
    tx_formatted_date = tx_datetime.strftime("%Y-%m-%d %H:%M:%S")
    unix_timestamp = int(time.mktime(tx_datetime.timetuple()))
    
    next_tx = {'datetime': tx_formatted_date, 'unixtimestamp': unix_timestamp, 'network': "Ripple"}

    #Unique Hash
    next_tx['txhash'] = i['hash']
    
    #Transaction Type
    transaction_type = i['tx']['TransactionType']
    next_tx['txtype'] = transaction_type
    
    #Fee
    next_tx['txfee'] = '{0:f}'.format(float(i['tx']['Fee'])/float(1000000))
    next_tx['txfeedenom'] = 'XRP'
    
    if transaction_type == "OfferCreate":      
        
        #Sender Pays in not XRP, and receives not XRP
        if type(i['tx']["TakerGets"]) == type({}) and type(i['tx']["TakerPays"]) == type({}): 
            
            #Account Sending
            next_tx['sender'] = i['tx']['Account']

            #Amount Sent
            next_tx['valuesent'] = i['tx']['TakerGets']['value']

            #Amount Sent Denomination
            next_tx['valuesentdenom'] = i['tx']['TakerGets']['currency']

            #Account Receiving
            next_tx['receiver'] = "Unknown"

            #Amount Received
            next_tx['valuereceived'] = i['tx']['TakerPays']['value']

            #Received Denomination
            next_tx['valuereceiveddenom'] = i['tx']['TakerPays']['currency']
            
            
        #Sender Pays in XRP, but Receives not XRP    
        elif type(i['tx']["TakerGets"]) != type({}) and type(i['tx']["TakerPays"]) == type({}): 
            #Account Sending
            next_tx['sender'] = i['tx']['Account']

            #Amount Sent
            next_tx['valuesent'] = '{0:f}'.format(int(i['tx']['TakerGets'])/float(1000000))

            #Amount Sent Denomination
            next_tx['valuesentdenom'] = 'XRP'

            #Account Receiving
            next_tx['receiver'] = "Unknown"

            #Amount Received
            next_tx['valuereceived'] = i['tx']['TakerPays']['value']

            #Received Denomination
            next_tx['valuereceiveddenom'] = i['tx']['TakerPays']['currency']
        
        #Sender Pays in not XRP and receives XRP
        elif type(i['tx']["TakerGets"]) == type({}) and type(i['tx']["TakerPays"]) != type({}): 
            #Account Sending
            next_tx['sender'] = i['tx']['Account']

            #Amount Sent
            next_tx['valuesent'] = i['tx']['TakerGets']['value']

            #Amount Sent Denomination
            next_tx['valuesentdenom'] = i['tx']['TakerGets']['currency']
            
            #Account Receiving
            next_tx['receiver'] = "Unknown"

            #Amount Received
            next_tx['valuereceived'] = '{0:f}'.format(int(i['tx']['TakerPays'])/float(1000000))

            #Received Denomination
            next_tx['valuereceiveddenom'] = 'XRP'
    
        #Sender Pays in XRP and Receives XRP
        else:

            #Account Sending
            next_tx['sender'] = i['tx']['Account']

            #Amount Sent
            next_tx['valuesent'] = '{0:f}'.format(int(i['tx']['TakerGets'])/float(1000000))

            #Amount Sent Denomination
            next_tx['valuesentdenom'] = 'XRP'

            #Account Receiving
            next_tx['receiver'] = "Unknown"

            #Amount Received
            next_tx['valuereceived'] = '{0:f}'.format(int(i['tx']['TakerPays'])/float(1000000))

            #Received Denomination
            next_tx['valuereceiveddenom'] = 'XRP'

    elif transaction_type == 'Payment':
            next_tx['sender'] = i['tx']['Account']

            #Amount Sent
            next_tx['valuesent'] = '{0:f}'.format(int(i['tx']['Amount'])/float(1000000))

            #Amount Sent Denomination
            next_tx['valuesentdenom'] = 'XRP'

            #Account Receiving
            next_tx['receiver'] = i['tx']['Destination']

            #Amount Received
            next_tx['valuereceived'] = '{0:f}'.format(int(i['tx']['Amount'])/float(1000000))

            #Received Denomination
            next_tx['valuereceiveddenom'] = 'XRP'
            
    processed_txs.append(next_tx)

In [166]:
len(processed_txs)

100

In [167]:
transactions[50]

{u'date': u'2017-12-31T00:00:01+00:00',
 u'hash': u'052F6B9D7CCF20B8372C133CBEDB57B43E5E31D1D689F5EF95E6346AADBD6F6D',
 u'ledger_hash': u'67231C0BD2074BAE003DA4084FFC117902C2DFA903A19F11246E04304C3AD010',
 u'ledger_index': 35444820,
 u'meta': {u'AffectedNodes': [{u'CreatedNode': {u'LedgerEntryType': u'DirectoryNode',
     u'LedgerIndex': u'11EEBF7DFC0076D299322039F1493C921CAAEFE85B322E5D5107FFDDBB8B2D9C',
     u'NewFields': {u'ExchangeRate': u'5107FFDDBB8B2D9C',
      u'RootIndex': u'11EEBF7DFC0076D299322039F1493C921CAAEFE85B322E5D5107FFDDBB8B2D9C',
      u'TakerPaysCurrency': u'0000000000000000000000004A50590000000000',
      u'TakerPaysIssuer': u'6F2531F2B8CDB96D6D986D9D75CC0156DF2C5387'}}},
   {u'DeletedNode': {u'FinalFields': {u'ExchangeRate': u'5107FFE837C35682',
      u'Flags': 0,
      u'RootIndex': u'11EEBF7DFC0076D299322039F1493C921CAAEFE85B322E5D5107FFE837C35682',
      u'TakerGetsCurrency': u'0000000000000000000000000000000000000000',
      u'TakerGetsIssuer': u'000000000000

In [168]:
processed_txs[50]

{'datetime': '2017-12-31 00:00:01',
 'network': 'Ripple',
 'receiver': 'Unknown',
 'sender': u'rhS2H7ETM3wBkFETvYycoUm9FEDYi44Pg4',
 'txfee': '0.000100',
 'txfeedenom': 'XRP',
 'txhash': u'052F6B9D7CCF20B8372C133CBEDB57B43E5E31D1D689F5EF95E6346AADBD6F6D',
 'txtype': u'OfferCreate',
 'unixtimestamp': 1514696401,
 'valuereceived': u'22516.5263629046',
 'valuereceiveddenom': u'JPY',
 'valuesent': '100.000000',
 'valuesentdenom': 'XRP'}

In [153]:
for n, i in enumerate(transactions):
    
    print i['tx']['TransactionType'], n

OfferCreate 0
OfferCreate 1
OfferCancel 2
OfferCreate 3
OfferCancel 4
OfferCreate 5
OfferCreate 6
OfferCancel 7
OfferCancel 8
OfferCancel 9
OfferCancel 10
OfferCreate 11
OfferCreate 12
OfferCreate 13
OfferCancel 14
OfferCreate 15
OfferCreate 16
OfferCreate 17
Payment 18
OfferCreate 19
OfferCreate 20
OfferCreate 21
OfferCancel 22
OfferCreate 23
OfferCreate 24
OfferCreate 25
OfferCreate 26
OfferCreate 27
OfferCreate 28
OfferCancel 29
OfferCreate 30
OfferCreate 31
OfferCreate 32
OfferCreate 33
OfferCreate 34
OfferCreate 35
OfferCreate 36
OfferCreate 37
OfferCreate 38
OfferCreate 39
OfferCreate 40
OfferCreate 41
OfferCreate 42
OfferCreate 43
OfferCreate 44
OfferCreate 45
OfferCreate 46
OfferCancel 47
OfferCreate 48
OfferCreate 49
OfferCreate 50
OfferCreate 51
OfferCreate 52
OfferCreate 53
OfferCreate 54
OfferCancel 55
OfferCancel 56
OfferCancel 57
OfferCreate 58
OfferCreate 59
OfferCancel 60
OfferCreate 61
OfferCreate 62
OfferCreate 63
OfferCreate 64
OfferCreate 65
SetRegularKey 66
OfferCr

In [170]:
df = pandas.DataFrame.from_dict(processed_txs)

df.to_csv("Ripple_20171231_transactions.csv")